In [ ]:
import pymc as pm
import arviz as az
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Problem 1

In [ ]:
n_samples = 500

freq = np.random.choice([25, 33, 50, 66], size=n_samples)
hd_size = np.random.choice([200, 340, 540, 850], size=n_samples)

true_alpha = 1500
true_beta1 = 15
true_beta2 = 200
true_sigma = 300

log_hd = np.log(hd_size)

mu = true_alpha + true_beta1 * freq + true_beta2 * log_hd
price = np.random.normal(mu, true_sigma)

df = pd.DataFrame({
    'price': price,
    'speed': freq,
    'hd': hd_size
})

In [ ]:
y_obs = df['price'].values
x1 = df['speed'].values # Processor frequency
x2 = np.log(df['hd'].values) # Natural log of HDD size

print(f"Data Loaded. Samples: {len(df)}")
print(f"Variables prepared: x1 (freq), x2 (log_hd), y (price)")

### solution a)

In [ ]:
with pm.Model() as model:
    alpha = pm.Normal('alpha', mu=0, sigma=1000)
    beta1 = pm.Normal('beta1', mu=0, sigma=100)
    beta2 = pm.Normal('beta2', mu=0, sigma=100)
    sigma = pm.HalfNormal('sigma', sigma=500)

    mu = alpha + beta1 * x1 + beta2 * x2
    obs = pm.Normal('obs', mu=mu, sigma=sigma, observed=y_obs)
    trace = pm.sample(2000, return_inferencedata=True)

### solution b)

In [ ]:
# Summary statistics containing HDI
summary = az.summary(trace, var_names=['beta1', 'beta2'], hdi_prob=0.95)
print(summary[['mean', 'sd', 'hdi_2.5%', 'hdi_97.5%']])

# Visual plot
az.plot_posterior(trace, var_names=['beta1', 'beta2'], hdi_prob=0.95, ref_val=0)
plt.show()

### solution c)

In [ ]:
hdi_beta1 = az.hdi(trace.posterior['beta1'], hdi_prob=0.95)
hdi_beta2 = az.hdi(trace.posterior['beta2'], hdi_prob=0.95)

print(f"Beta1 95% HDI: {hdi_beta1.values}")
print(f"Beta2 95% HDI: {hdi_beta2.values}")

### Solution d)

In [ ]:
new_freq = 33
new_hd = 540
new_x2 = np.log(new_hd)

In [ ]:
post = trace.posterior
alpha_samples = post['alpha'].values.flatten()
beta1_samples = post['beta1'].values.flatten()
beta2_samples = post['beta2'].values.flatten()

mu_samples = alpha_samples + (beta1_samples * new_freq) + (beta2_samples * new_x2)

In [ ]:
mu_hdi = az.hdi(mu_samples, hdi_prob=0.90)

print(f"Expected Mean Price (mu): {mu_samples.mean():.2f}")
print(f"90% HDI for Mean Price: {mu_hdi}")

### solution e)

In [ ]:
sigma_samples = post['sigma'].values.flatten()

y_pred_samples = np.random.normal(loc=mu_samples, scale=sigma_samples)

y_hdi = az.hdi(y_pred_samples, hdi_prob=0.90)

print(f"Predicted Individual Price: {y_pred_samples.mean():.2f}")
print(f"90% HDI for Predicted Price: {y_hdi}")

plt.figure(figsize=(10, 5))
plt.hist(mu_samples, bins=50, alpha=0.7, label='Expected Price (Mean)', density=True)
plt.hist(y_pred_samples, bins=50, alpha=0.5, label='Predicted Price (Individual)', density=True)
plt.title('Comparison: Uncertainty in Mean vs. Uncertainty in Prediction')
plt.legend()
plt.show()